In [1]:
import os
#os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
#os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0
#import sys
#sys.path.append('/content/gdrive/S:/OSSP-master/CJ')
import tkinter as tk
from tkinter import *
import tkinter.font as font
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import matplotlib.pyplot as plt
import numpy as np
import h5py
import argparse
from ctypes import *
from osgeo import gdal
import tqdm
from select import select
from lib import utils # you have to upload to runtime session in google collab during every session
import skimage
from skimage.segmentation import mark_boundaries
from lib import preprocess as pp
from lib.segment import segment_image
#from segment2 import segment_image2 as segment_image
from lib import attribute_calculations as attr_calc
#import rasterio
#import rasterio.features
#import rasterio.warp
#from geojson import Feature, Point, FeatureCollection
import createGeoJson as GJ


#Global vars
MOUSE_X = 0
MOUSE_Y = 0
GAUSS_SIGMA = 2
FEATURE_SEPARATION = 5

#Colors

#font1 = font.Font(family = 'helvetica')

class colorPalette:
    def __init__(self, primaryColor, secondaryColor, tertiaryColor):
        self.primaryColor = primaryColor
        self.secondaryColor = secondaryColor
        self.tertiaryColor = tertiaryColor
    #!TODO: (if/when time permits) add different color pallettes
    
lightMode = colorPalette('#e3e3fc', '#f7f7ff', 'black')
#darkMode = colorPalette( , , ,)

# Buttons

In [2]:
class ButtonsLeft(tk.Frame): #Class for all of the buttons of the GUI
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.config(bg = lightMode.secondaryColor)
        self.parent = parent
        
        # Slider that lets us traverse an image by its x and y coordinates
        self.yNavSlider = tk.Scale(self, from_ = 0, to_ = 0, length = 160, bg = lightMode.primaryColor, orient = VERTICAL, relief = RAISED,
                             command = self.getYNavVal)
        self.yNavSlider.grid(column = 0, row = 0, pady = (0,20), sticky = 'ns')
        
        # Instead of radio buttons we will attempt to use two sliders - one for each axis of the image
        self.xNavSlider = tk.Scale(self, from_ = 0, to_ = 0, length = 160, bg = lightMode.primaryColor, orient = HORIZONTAL,
                             command = self.getXNavVal)
        self.xNavSlider.grid(column = 0, row = 1, pady = (0,20), sticky = 'ns')

        # Radio buttons that allow our one dimensional image navigator to switch which dimension it controls
#        radioVar = tk.IntVar(value = 1) # Create variable to mark which mode the slider is in
#        xRadioButton = tk.Radiobutton(self, text = "X", variable = radioVar, value = 1, bg = lightMode.secondaryColor, padx = 5)
#        xRadioButton.grid(column = 0, row = 1, padx = (0,40), sticky = 'ns')
#        yRadioButton = tk.Radiobutton(self, text = "Y", variable = radioVar, value = 2, bg = lightMode.secondaryColor, padx = 5)    
#        yRadioButton.grid(column = 0, row = 1, padx = (40,0), sticky = 'ns')
        
        
        dataLabel = tk.Label(self, text = "Training Data", width = 18, height = 2)
        dataLabel.grid(column = 0, row = 2, sticky = 'nsew')
        
        button0 = tk.Button(self, text = "Save", bg = lightMode.primaryColor, width = 9, height = 2)
        button0.grid(column = 0, row = 3, sticky = 'nsw')
        
        button1 = tk.Button(self, text = "Save As", bg = lightMode.primaryColor, width = 9, height = 2)
        button1.grid(column = 0, row = 3, sticky = 'nse')
        
        geoLabel = tk.Label(self, text = "GeoJSON Data", width = 18, height = 2)
        geoLabel.grid(column = 0, row = 4, pady = (10, 1), sticky = 'nsew')
        
        button2 = tk.Button(self, text = "Save", bg = lightMode.primaryColor, width = 9, height = 2,
                           command = lambda: parent.data.createNewTds())
        button2.grid(column = 0, row = 5, sticky = 'nsw') 
        
        button3 = tk.Button(self, text = "Save As", bg = lightMode.primaryColor, width = 9, height = 2,
                           command = lambda: parent.data.createNewTds())
        button3.grid(column = 0, row = 5, sticky = 'nse') 
        
        button4 = tk.Button(self, text = "Splice Image", bg = lightMode.primaryColor, width = 18, height = 2)
        button4.grid(column = 0, row = 6, pady = (15,1), sticky = 'nsew')
        
    def getYNavVal(self, newVal):
        self.yNav = newVal 
        try:
            self.parent.imageDisplay.updateImages(MOUSE_X, MOUSE_Y, yNav = self.yNav)
        except TypeError as error:
            pass
    def getXNavVal(self, newVal):
        self.xNav = newVal
        try:
            self.parent.imageDisplay.updateImages(MOUSE_X, MOUSE_Y, xNav = self.xNav)
        except TypeError as error:
            pass
    
    

In [3]:
class ButtonsBottom(tk.Frame): #!TODO fix doccumentation for the button classes to better explain code
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.config(bg = lightMode.primaryColor)
  
        waterClassButton = tk.Button(self, text = "Water", bg = '#03fcec', fg = 'white', width = 16, height = 2,
                                    command = lambda: parent.event.classify('water') )
        waterClassButton.grid(column = 1, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        thinIceClassButton = tk.Button(self, text = "Thin Ice", bg = '#03dbfc', fg = 'white',width = 16, height = 2,
                                    command = lambda: parent.event.classify('gray') )
        thinIceClassButton.grid(column = 2, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        shadowClassButton = tk.Button(self, text = "Shadow", bg = 'black', fg = 'white',width = 16, height = 2,
                                    command = lambda: parent.event.classify('shadow') )
        shadowClassButton.grid(column = 3, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        #!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ADD FUNCTIONALITY TO THIS BUTTON -BROKEN
        subIceClassButton = tk.Button(self, text = "Submerged Ice", bg = '#038cfc', fg = 'white',width = 16, height = 2,
                                    command = lambda: parent.event.classify('subIce') )
        subIceClassButton.grid(column = 4, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        snowClassButton = tk.Button(self, text = "Ice/Snow", bg = '#FFFFFF', fg = 'black',width = 16, height = 2,
                                    command = lambda: parent.event.classify('snow') )
        snowClassButton.grid(column = 5, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        meltPondClassButton = tk.Button(self, text = "Melt Pond", bg = 'pink', fg = 'white',width = 16, height = 2,
                                    command = lambda: parent.event.classify('Melt==') )
        meltPondClassButton.grid(column = 6, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        

In [4]:
class ButtonsRight(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.config(bg = lightMode.secondaryColor)
        self.parent = parent
        self.zoom = 50
        
        #Zoom Slider: allows us to zoom in on an image
        zoomSlider = tk.Scale(self, from_ = 400, to = 0, length = 160, bg = lightMode.primaryColor, orient = VERTICAL, relief = RAISED,
                             command = self.getZoomSliderVal)#!TODO: command = self.getZoomSliderVal)
        zoomSlider.grid(column = 0, row = 0, pady = (0,20), sticky = 'ns')
                 
        zoomLabel = tk.Label(self, text = "Zoom +-", bg = lightMode.secondaryColor)
        zoomLabel.grid(column = 0, row = 1, pady = (0,2), sticky = 'nsew')
        
        
        nextImageButton = tk.Button(self, text = ">", bg = lightMode.primaryColor, width = 9, height = 2,
                                   command = lambda: parent.event.nextImage())
        nextImageButton.grid(column = 0, row = 3, sticky = 'nse')
        
        prevImageButton = tk.Button(self, text = "<",  bg = lightMode.primaryColor, width = 9, height = 2,
                                   command = lambda: parent.event.prevImage())
        prevImageButton.grid(column = 0, row = 3, sticky = 'nsw')
        
        # Initialize Image Button
        initImageButton = tk.Button(self, text = "Initialize Image", bg = "yellow", width = 18, height = 2,
                                   command = lambda: parent.event.initializeImage())
        #initImageButton['font'] = font1
        initImageButton.grid(column = 0, row = 2, pady = (20,1), sticky = 'nsw')
        
    def getZoomSliderVal(self, newVal):
        self.zoom = newVal
        try:
            self.parent.imageDisplay.updateImages(MOUSE_X, MOUSE_Y)
        except TypeError as error:
            pass

# Image Display

In [5]:
class ImageDisplay(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        
        self.displayImage = None
    
        self.disp_xdim , self.disp_ydim, = 0, 0
        self.zoom_win_x, self.zoom_win_y = 0, 0

        #Image Canvas
        self.graph = plt.figure(figsize = [8, 4]) # width x height
        self.graph.subplots_adjust(left = 0.01, right = 0.99,
                                 bottom = 0.05, top = 0.99,
                                 wspace = 0.01, hspace = 0.01)
        imageCanvas = FigureCanvasTkAgg(self.graph, self)
        #self is the Tk canvas that we are adding self.graph to
        imageCanvas.draw()
        imageCanvas.get_tk_widget().grid(column = 2, row = 2) #Plot the canvas as a widget
        #!TODO: add functionalilty with the event manager
        
        self.homescreen()
        
    def loadingScreen(self):
        plt.clf()
        loadingText = "Images are loading, please wait... "
        ax = self.graph.add_subplot(1, 1, 1, adjustable='datalim', frame_on=False)
        ax.text(0.5, 0.5, loading_text, horizontalalignment='center', verticalalignment='center')
        #!TODO: add turtle graphics loading graphic
        ax.axis('off')

        self.graph.canvas.draw()
    
    
    def homescreen(self): #The screen the user will be met with upon opening the GUI
        plt.clf()
        openingMsg = "Welcome to ArcCI semantic segmentation training \n Press <Initialize Image> to begin."
        
        openingScreen = self.graph.add_subplot(1, 1, 1)
        openingScreen.text(.5, .55, openingMsg, horizontalalignment = 'center', verticalalignment = 'top', weight = 'bold')
        openingScreen.axis('off')
        
        self.graph.canvas.draw()  
        
            
    def initializeImage(self):
        if self.parent.data.imgType == 'wv02_ms':
            self.displayImage = utils.create_composite([self.parent.data.originalImage[4, :, :],
                                                         self.parent.data.originalImage[2, :, :],
                                                         self.parent.data.originalImage[1, :, :]],
                                                         dtype=np.uint8)

        elif self.parent.data.imgType == 'pan':
            self.displayImage = utils.create_composite([self.parent.data.originalImage[0, :, :],
                                                         self.parent.data.originalImage[0, :, :],
                                                         self.parent.data.originalImage[0, :, :]],
                                                         dtype=np.uint8)

        elif self.parent.data.imgType == 'srgb':
            self.displayImage = utils.create_composite([self.parent.data.originalImage[0, :, :],
                                                         self.parent.data.originalImage[1, :, :],
                                                         self.parent.data.originalImage[2, :, :]],
                                                         dtype=np.uint8)
                        
        self.disp_xdim, self.disp_ydim = np.shape(self.displayImage)[0:2]

    def loadingScreen(self):
        #This function displays a loading screen between uploading images
        plt.clf()
        loadingText = "Images are loading, please wait..."
        ax = self.graph.add_subplot(1, 1, 1, adjustable = 'datalim', frame_on = False)
        ax.text(0.5, 0.5, loadingText, horizontalalignment = 'center', verticalalignment = 'center')
        ax.axis('off')
        
        self.graph.canvas.draw()
        
    def updateImages(self, x, y, **kwargs):
        
        global gauss_sigma0, feature_separation0
        
        gauss_sigma0 = 3
        feature_separation0 = 1
        xNav = kwargs.get("xNav", None)
        yNav = kwargs.get("yNav", None)
        
        
        plt.clf()

        zoom_size = int(self.parent.rButtons.zoom)
        
        x_min = y                     # x_min: row, is onclick ydata 
        x_max = y + (zoom_size * 2)   # x_max: row, is onclick ydata
        y_min = x                     # y_min: col, is onclick xdata 
        y_max = x + (zoom_size * 2)   # y_max: col, is onclick xdata 

        # Store the zoom window corner coordinates for reference in onclick()
        # xMin and yMin are defined backwards
        if yNav is not None:
            self.zoom_win_x = y_min + int(yNav)
        if xNav is not None:
            self.zoom_win_y = x_min + int(xNav)
        #for nav slider ^
        
        x_min += self.zoom_win_x
        x_max += self.zoom_win_x
        y_min += self.zoom_win_y
        y_max += self.zoom_win_y
        
        if x_min < 0:
            x_min = 0
            x_max = zoom_size*2-1
        if x_max >= self.disp_xdim:
            x_max = self.disp_xdim - 1
            x_min = x_max- zoom_size*2
        if y_min < 0:
            y_min = 0
            y_max = zoom_size*2-1
        if y_max >= self.disp_ydim:
            y_max = self.disp_ydim - 1
            y_min = y_max- zoom_size*2 
            
        if x_min == x_max:
            x_max += 1
        if y_min == y_max:
            y_max += 1
            
        self.imageHeight = self.displayImage.shape[0]
        self.imageWidth = self.displayImage.shape[1]

        #print(self.displayImage.shape)
        croppedImage = self.displayImage[x_min:x_max, y_min:y_max]

        imageData = croppedImage
        
        if self.parent.data.imgType == 'pan':
            imageData=croppedImage[:,:,0]
            imageData=np.reshape(imageData, (1, imageData.shape[0], imageData.shape[1]))
        
        if self.parent.data.imgType == 'srgb':
            imageData=np.stack( (imageData[:,:,0], imageData[:,:,1], imageData[:,:,2]), axis=0)  #(row, col, 3)-> (3, row, col)
        
     
        if imageData.shape[0] > 1:
            segImage = segment_image(imageData, gauss_sigma0, feature_separation0, image_type=self.parent.data.imgType)


            if np.max(segImage) > 0:
                print(np.max(imageData))  

                watershedResult = mark_boundaries(imageData[0], segImage)
                watershedResult = mark_boundaries(croppedImage, segImage)  
                watershedResult = 255 * watershedResult
                watershedResult = watershedResult.astype(np.uint8) 

            else:
                watershedResult = np.zeros(np.shape(segImage)).astype(np.uint8)

            # Image 3 (Zoomed in image, with segment highlight)
            colorImage = watershedResult
    
            # Plotting onto the GUI
            ax = self.graph.add_subplot(2, 2, 1)
            ax.imshow(colorImage, interpolation='None', vmin=0, vmax=255)
            ax.tick_params(axis='both',  # changes apply to the x-axis
                          which='both',  # both major and minor ticks are affected
                           bottom=False,  # ticks along the bottom edge are off
                           top=False,  # ticks along the top edge are off
                           left=False,
                           right=False,
                           labelleft=False,
                           labelbottom=False)
            ax.set_label('ax1')

        ax = self.graph.add_subplot(1, 2, 2)
        ax.imshow(croppedImage, interpolation='None', vmin=0, vmax=255)
        ax.tick_params(axis='both',  # changes apply to the x-axis
                       which='both',  # both major and minor ticks are affected
                       bottom=False,  # ticks along the bottom edge are off
                       top=False,  # ticks along the top edge are off
                       left=False,
                       right=False,
                       labelleft=False,
                       labelbottom=False)
        ax.set_label('ax2')

        ax = self.graph.add_subplot(1, 2, 1)
        ax.imshow(self.displayImage, interpolation='None', vmin=0, vmax=255)
        ax.axvspan(y_min,
                   y_max,
                   1. - float(x_max) / self.disp_xdim,
                   1. - float(x_min) / self.disp_xdim,
                   color='red',
                   alpha=0.3) 
        ax.set_xlim([0, np.shape(self.displayImage)[1]])
        ax.tick_params(axis='both',  # changes apply to the x-axis
                       which='both',  # both major and minor ticks are affected
                       bottom=False,  # ticks along the bottom edge are off
                       top=False,  # ticks along the top edge are off
                       left=False,
                       right=False,
                       labelleft=False,
                       labelbottom=False)
        #ax.set(facecolor = "pink")
        ax.set_label('ax3')


        # Updating the plots
        self.graph.canvas.draw()

# Training Window

In [6]:
class TrainingWindow(tk.Frame):

#    def __init__(self, parent, img_list, tds_filename, username, im_type, activate_autorun=False):
    def __init__(self, parent, imgList, tdsFilename, imgType, rasterList, activate_autorun = False):#!, img_list, username, im_type, activate_autorun=False):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.parent.title("ArcCI Deep Learning Training GUI")

        
        #!TODO create custom icon: self.parent.iconbitmap('vectorstock_7199163_icon.ico') # Designed by freepik (Image #7199163 at VectorStock.com)
        
        # the bottom and right buttons are declared before the left buttons
        # the left buttons require that we augment the nav sliders to match the size of any given image using gdal metadata.
    
        self.classificationButtons = ButtonsBottom(self)
        self.classificationButtons.grid(column = 1, row = 1, columnspan = 3, sticky="NEW")
        
        self.rButtons = ButtonsRight(self)
        self.rButtons.grid(column = 2, row = 0, rowspan = 3, sticky="NSEW")

        #Data manager object
        self.data = DataManager(imgList, imgType, tdsFilename, rasterList)
        self.data.loadTrainingData()
        
        self.lButtons = ButtonsLeft(self)
        self.lButtons.grid(column = 0, row = 0, rowspan = 3, sticky="NSEW") 
    
        # Image display window
        self.imageDisplay = ImageDisplay(self)
        self.imageDisplay.grid(column = 1, row = 0)
        
        # Manager for all the GUI events (e.g. button presses)
        self.event = EventManager(self)    # Define event_manager functions, should be found in EventManager()
        
        
        #self.progress_bar = ProgressBar(self)
        #self.progress_bar.grid(column=1, row=0)   # Progress info shown in col=1, row=0 (top right corner)

        #self.progress_bar.update_progress()
        
        #!TODO: add live prediction object

    def exitGui(self):
        self.parent.quit()
        self.parent.destroy()
        
def calculateAttributes(originalImage, secondaryImage,
                       wb_ref, br_ref, imgDate, segmentID, imgType):
    featureArray = []
    
    if imgType == 'pan':
        feature_array = attr_calc.analyze_pan_image(originalImage,
                                                    secondaryImage,
                                                    imgDate,
                                                    segmentID=segmentID)
    if imgType == 'srgb':
        feature_array = attr_calc.analyze_srgb_image(originalImage,
                                                     secondaryImage,
                                                     segmentID=segmentID)
    if imgType == 'wv02_ms':
        feature_array = attr_calc.analyze_ms_image(originalImage,
                                                   secondaryImage,
                                                   wb_ref,
                                                   br_ref,
                                                   segmentID=segmentID)
    return feature_array
    
def getImages(segmentList):
    imageList = []
    for seg_id in segmentList:
        if not seg_id[0] in imageList:
            imageList.append(seg_id[0])
    return imageList

def validateTds(tdsFilename, inputDir, imgType):
    #If there is not a tdsFilename provided, we will create a default here
    if tdsFilename is None:
        tdsFilename = os.path.join(inputDir, imgType + "_training_data.h5")
    elif os.path.isfile(tdsFilename): #If there was a file given
        try:
            dataFile = h5py.File(tdsFilename, 'r')
            dataFile.close()
        except OSError:
            print("Invalid Data File.")
            quit()
    return tdsFilename
        
# Finds all the unique images from the given directory
def scrapeDir(src_dir):
    imageList = []

    for ext in utils.valid_extensions:   # valid_extensions is a list ['.tif','.tiff','.jpg']
        rawList = utils.get_image_paths(src_dir, keyword=ext) # utils.py module
        for rawImg in rawList:
            imageList.append(rawImg)

    # Save only the unique entries
    imageList = list(set(imageList))
    utils.remove_hidden(imageList)

    return imageList

# Data Manager

In [7]:
class DataManager: 
    def __init__(self, availableImages, imgType, tdsFilename, rasterList):
        # Image and segment data (populated in load_image())
        self.originalImage = None
        self.segmentedImage = None

        # Variable Values   (populated in load_training_data())
        self.labelVector = []
        self.segmentList = []
        self.featureMatrix = []
        self.tracker = 0   # Number of segment sets added from the current image
        self.imageIndex = 0   # Index for progressing through available images
        
        # GeoJSON Converter
        self.GJ = GJ.geoJsonConverter()
        #For GeoJSON conversion - populated in _loadImage()
        self.prevImageData = None
        self.newFileNameIndex = 1
        
        # Global Static Values
        self.tdsFilename = tdsFilename
        self.imgType = imgType
        self.availableImages = availableImages
        self.rasterList = rasterList

        #self.configList = [] # Creates list to hold indexes of images... event.i replaces this - do i need?
        #self.configIndex = 0 # allowing us to configure sliders appropriately
        #for img in self.availableImages:
            #self.configList.append(self.configIndex)  # 
            #self.configIndex += 1
        
        # Image Static Value (populated in load_image())
        self.wb_ref = None
        self.br_ref = None
        self.imgDate = None
        self.imgName = None
        self.watershed_result = None
        
    def getImageIndex(self):
        return self.imageIndex
    
    def getSizeLists(self):
        imgWidthList = []
        imgHeightList = []
        i = 0 # indexing var
        while i < len(self.rasterList):
            imgWidthList.append(self.rasterList[i].RasterXSize)
            imgHeightList.append(self.rasterList[i].RasterYSize)
            i += 1
        return imgWidthList, imgHeightList
    
    def loadNextImage(self):
        # Increment the image index
        self.imageIndex += 1
        # Loop im_index based on the available number of images
        self.imageIndex = self.imageIndex % len(self.availableImages)
        # Load the new data
        self._loadImage()


    def loadPreviousImage(self):
        # Increment the image index
        self.imageIndex -= 1
        # Loop im_index based on the available number of images
        self.imageIndex = self.imageIndex % len(self.availableImages)
        # Load the new data
        self._loadImage()

    def _loadImage(self):
        # Loads the optical and segmented image data from disk. Should only be called from
        #   loadNextImage an loadPreviousImage
        fullImageName = self.availableImages[self.imageIndex]

        self.imageName = os.path.splitext(os.path.split(fullImageName)[1])[0]

        src_ds = gdal.Open(fullImageName, gdal.GA_ReadOnly)

        # Read the image date from the metadata
        metadata = src_ds.GetMetadata()
        self.imgDate = pp.parse_metadata(metadata, self.imgType)

        # Determine the datatype
        src_dtype = gdal.GetDataTypeSize(src_ds.GetRasterBand(1).DataType)

        # Calculate the reference points from the image histogram
        lower, upper, wb_ref, br_ref = pp.histogram_threshold(src_ds, src_dtype)
        self.wb_ref = np.array(wb_ref, dtype=c_uint8)
        self.br_ref = np.array(br_ref, dtype=c_uint8)
        
        # Load the image data
        imageData = src_ds.ReadAsArray()     # <class 'numpy.ndarray'> uint8 (3, 7471, 8242)
        #print(imageData.shape) #debug
        if imageData.ndim == 2:
            imageData = imageData.reshape((1,imageData.shape[0],imageData.shape[1]))
        #print(imageData.shape) #debug
         #Close the GDAL dataset
        src_ds = None

        # Rescale the input dataset using a histogram stretch
        imageData = pp.rescale_band(imageData, lower, upper)
        #print(imageData.shape) #debug      
        # Apply a white balance to the image
        imageData = pp.white_balance(imageData, self.wb_ref.astype(np.float64), float(np.amax(self.wb_ref)))
        #print(imageData.shape) #debug
        # Convert the input data to c_uint8
        self.originalImage = np.ndarray.astype(imageData, c_uint8) # Necessary
        # print(self.originalImage) #debug - fixed
        self.prevImageData = self.originalImage
        #print("Creating segments on provided image...")
 # keep       #watershedImage = segment_image(imageData, gauss_sigma0 = 1, feature_separation0 = 2, image_type = self.imgType)
        #watershedImage = segment_image(imageData, smooth = 100, gradient_cut = 100, merging_cut = 100, image_type = self.imgType)
        #def segment_image2(input_data, smooth, gradient_cut, merging_cut, image_type=False):
        # Convert the segmented image to c_int datatype. This is needed for the
        # Cython methods that calculate attribute of segments.
# keep       #self.segmentedImage = np.ndarray.astype(watershedImage, c_uint32)
        #print(watershedImage.shape)
        #print(self.segmentedImage.shape) #They are the same shape- watershed is created incorrectly
        
        imageData = None
        watershedImage = None
        
    def loadTrainingData(self):
        try:
            with h5py.File(self.tdsFilename, 'r') as dataFile:
                   # Load the existing feature matrix and segment list if they exist,
                   #   otherwise initialize an empty array for these lists.
                if 'featureMatrix' in list(dataFile.keys()):
                    self.featureMatrix = dataFile['featureMatrix'][:].tolist()
                else:
                    self.featureMatrix = []

                if 'segmentList' in list(dataFile.keys()):
                    # For loading files created in py2
                    self.segmentList = [[name[0].decode(), name[1].decode()] for name in dataFile['segmentList']]
                else:
                    self.segmentList = []
              # If the file does not exist, create empty values
        except OSError:
            self.featureMatrix = []
            self.segmentList = []
            self.labelVector = []   
           
    def getNumLabels(self):
        return len(self.labelVector)
    
    def appendLabel(self, label):
        self.tracker += 1
        self.labelVector.append(label)
        
    def removePrevLabel(self):
        self.labelVector.pop()
        self.tracker -= 1
        
    def getNumSegments(self):
        return len(self.segmentList)
    
    def getCurrentSegment(self):
        return self.segmentList[len(self.labelVector)]
    
    def addSingleSegment(self, newSegment):
        self.segmentList.append(newSegment)
        
    def trimSegmentList(self):
        self.segmentList = self.segmentList[:len(self.labelVector)]
        
    #adds segment to list of segments to be classified
    def addSegments(self):
        segmentsToAdd = []
        #print(self.segmentedImage)
        a = 0
        # Select random x,y coordinates from the input image, and pick the segment where the random
        #   pixel lands. This makes the selected segments representative of the average surface
        #   distribution within the image. This still wont work if the image has a minority of any
        #   particular surface type.
        while len(segmentsToAdd) < 10:
            a += 1
            z, x, y = np.shape(self.originalImage)
            i = np.random.randint(x)
            j = np.random.randint(y)
            
            #print(self.segmentedImage[i])
            #print(self.segmentedImage[j])
            
            segmentID = self.segmentedImage[i][j]
            spSize = np.sum(self.segmentedImage == segmentID)
            if spSize >= 20:
                #checks for duplicate segment
                newSegment = [self.imgName,
                             "{}".format(segmentID)]
                if newSegment not in self.segmentList and newSegment not in segmentsToAdd:
                    segmentsToAdd.append(newSegment)
        print(("Attempts: {}".format(a)))
        self.segmentList += segmentsToAdd
        
    def computeAttributes(self, segmentID):
        #Create the list to store the attributes of the segment
        fetureArray = calcAttributes(self.originalImage, self.segmentedImage,
                   self.wb_ref, self.br_ref, self,imgDate, segmentID, self.imgType)
        #Convert feature matrix to feature array
        featureArray = featureArray[0]
        
        return featureArray
    
    def appendFeatures(self, featureArray):
        # If there are fewer features than labels, assume the new one should be appended
        # to the end    
        if len(self.featureMatrix) == len(self.labelVector) - 1:
            self.featureMatrix.append(featureArray)
        else:
            self.featureMatrix[len(self.labelVector) - 1] = featureArray
            
    def createGeoJSONObject(self):
    #Creates GeoJSON object        
        #conversion.setEncodingImg(self.parent.data.prevImageData)
        #self.parent.GJ.setEncodingImage(self.parent.data.prevImageData) #Set displayed image to be encoded in JSON
        JSONobject = self.GJ.imgToJson(self.prevImageData) # Create JSON object from encoded image
        print(JSONobject) # debug
        #GEOJSONobject = self.parent.GJ.JsonToGeoJson(JSONobject) # Convert JSON object to GeoJSON
        #preEncodeImage = self.parent.data.prevImageData.tolist()
        #print(self.prevImageData) # debug -fixed
        #print(preEncodeImage)
        #self.parent.GJ.imgToJson() #!TODO: finish step to GeoJson object  ^^ 1) remove print in GJ 2) add 2GeoJson method w/ relevant metadata
    def createNewTds(self):
    #(doesn't yet) Creates a new .h5 training file (like COCO) !TODO:
    #(does) pass current .h5 data to a geojson object
        

#Format name for new file
        while True: # infinite loop
            indexToCat = str(self.newFileNameIndex) # Convert our current file index to string to allow concatenation
            newFileName = 'S:\ArcCi\\ArcCI-Collab-Repo\\tdsObjects' + indexToCat + '.geojson' # concatenate target directory with file index
            self.newFileNameIndex += 1 #increment the file index to allow us to create a new file in each iteration
            break # break infinite loop
            
        #self.outfile = outfile
        self.GJ.plugTrainingData(self.featureMatrix, self.labelVector, self.segmentList, newFileName) #send training data to new GeoJSON object

# Event Manager

In [8]:
class EventManager: #!
    def __init__(self, parent):
        self.parent = parent
        self.is_ctive = False #Keeps events from happening when images are loading.
        #self.beacon = False # beacon that requests reconfig - necessary?
        self.gate = True # Gate that allows us to initialize an index variable for navigating images
        self.imgWidthList, self.imgHeightList = self.parent.data.getSizeLists() # Create size lists for nav config method
        
    def activate(self):
        self.is_active = True
        
    def deactivate(self):
        self.is_activte = False
        
    def nextSegment(self):
        if not self.is_active:
            return
        
        if self.parent.data.getNumLabels() == self.parent.data.getNumSegments():
                self.parent.data.addSegments()
                
                #TODO: add unet model here
                
        current_segment = self.parent.data.getCurrentSegment()
        segment_id = int(current_segment[1])
        
        self.parent.imageDisplay.updateImages(segment_id[0],segment_id[1])
        #self.parent.imageDisplay.updateImages(segment_id[0],segment_id[1])
        
    def previousSegment(self):
        if not self.is_active:
            return
        if self.parent.data.getNumLabels() == 0:
            return
        else:
            self.parent.data.removeLastLabel()
            current_segment = self.parent.data.getCurrentSegment()
            # loading bar here !TODO:
            
            if current_segment[0] != self.parent.data.imgName:
                self.previousImage()
                return
            segment_id = int(current_segment[1])
            
            self.parent.imageDisplay.updateImages(segment_id)
    
    def onclick(self, event):
        global MOUSE_X, MOUSE_Y
        if not self.is_active:
            return
        if event.inaxes is not None:
            axes_properties = event.inaxes.properties()
            x, y = 0, 0
            #If click was in image window
            if axes_properties['label'] == 'ax3': 
                x = int(event.xdata)
                y = int(event.ydata)
            #If the click was in the other windows
            if axes_properties['label'] == 'ax1' or axes_properties['label'] == 'ax2':
                win_x = int(event.xdata)
                win_y = int(event.ydata)
                x = self.parent.imageDisplay.zoom_win_x + win_x
                y = self.parent.imageDisplay.zoom_win_y + win_y
                
            MOUSE_X = x
            MOUSE_Y = y
            
            self.parent.imageDisplay.update_images(MOUSE_X, MOUSE_Y)
    
    def save(self):
        
        if self.parent.data.label_vector == []:
            return
        print("Saving Data...")
        
        previousNames = []
        previousData = []

        try:
            with h5py.File(self.parent.data.tdsFilename, 'r') as infile0:
            # Compiles all of the user data that was in the previous training validation file so that
            # it can be added to the new file as well. (Because erasing and recreating a .h5 is easier
            # than altering an existing one)
                for previousUser in list(infile.keys()):
                    if previousUser != 'feature_matrix' and previousUser != 'segment_list' and previousUser != username:
                        previousNames.append(previousUser)
                        previousData.append(infile[previousUser][:])
                infile.close()
        except OSError:
            pass
        
        # Change training datatset to reflect changes
        with h5py.File(self.parent.data.tdsFilename, 'w') as outfile:
            outfile.create_dataset('feature_matrix', data = self.parent.data.featureMatrix)
            

    def classify(self, key_press):
        if not self.is_active:
            return

        # Assigning the highlighted segment a classification
        segment_id = int(self.parent.data.getCurrentSegment()[1])
        print("Segment ID: {}".format(segment_id))
        # Note that we classified one more image
        if key_press == "snow":
            self.parent.data.appendLabel(1)
        elif key_press == "gray":
            self.parent.data.appendLabel(2)
        elif key_press == "melt":
            self.parent.data.appendLabel(3)
        elif key_press == "water":
            self.parent.data.appendLabel(4)
        elif key_press == "shadow":
            self.parent.data.appendLabel(5)
        elif key_press == "unknown":
            self.parent.data.appendLabel(6)

        # Calculate the attributes for the current segment
        feature_array = self.parent.data.computeAttributes(segment_id)
        self.parent.data.appendFeatures(feature_array)

        # Printing some useful statistics
        print("Assigned value: {} ({})".format(str(self.parent.data.label_vector[-1]), key_press))

        if self.parent.live_predictor.is_active():
            self.parent.live_predictor.print_prediction(feature_array)

        print(("~"*80))

        self.parent.progress_bar.update_progress()

        self.next_segment()
    #!TODO def autorun(self):
    
    def nextImage(self):
        #if not self.isActive:
            #`return 
        self.deactivate()
        # Remove unlabeled segments from segment list
        self.parent.data.trimSegmentList()
        # Save data
        #self.save() #!TODO: Fix self.save()
        # Set display to loading screen
        self.parent.after(10, self.parent.imageDisplay.loadingScreen())
        # Load next image data
        self.parent.data.loadNextImage()
        # move new data to display
        self.parent.imageDisplay.initializeImage()
        # Updates display, moves to next segment, adds segment to queue
        #!TODO self.parent.loadingBar.updateProgress()
        self.activate()
        
        # Navigation Slider Configuration Conditionals
        if self.i < len(self.parent.data.availableImages): # index is atleast n-1 of image array of size n
            self.i += 1 # increment the list for next image config
        if self.i >= len(self.parent.data.availableImages): # index is last image or has exceeded array
            self.i = 0 # reset config index to the begining of the image list
        
        #self.nextSegment()
        self.parent.imageDisplay.updateImages(0, 0)
        self.configNavSliders() # inform the mainloop to update the nav sliders
        
    def prevImage(self):
        self.deactivate()
        # Display to loading screen
        self.parent.after(10, self.parent.imageDisplay.loadingScreen())
        # load previous image data
        self.parent.data.loadPreviousImage()
        # add new data to display class
        self.parent.imageDisplay.initializeImage()
        # Updates display, moves to next segment, adds segment to queue
        #!TODO self.parent.loadingBar.updateProgress()
        self.activate()
        #self.nextSegment()
        self.parent.imageDisplay.updateImages(0, 0)
        
        # Navigation Slider Configuration Conditionals
        if self.gate == False:         # If the gate has been broken
            if self.i > 0: # if this is not the first image in the list
                self.i -= 1 # increment the list for last image config (forward if init_image is pressed twice)
            else: # If the index is zero and the user causes prevImage() to be called we will...
                self.i = len(self.parent.data.availableImages) # change the index of the list to the end of the image list
                                                                # allowing us to traverse the image array from both ends.
        if self.gate: # If the gate has yet to be broken
            self.i = 0
            self.gate = False # Break the one time gate
        
        
        self.configNavSliders() # inform the mainloop to update the nav sliders
        
    def initializeImage(self):
        if len(self.parent.data.availableImages) == 0:
            print("No images to load!")
            return
        #Make sure no image has been loaded already
        if self.parent.data.imgName is not None:
            return
        #prevImage() loads the first image as well
        if self.gate == False: # If uses presses init button a second time     
            self.i += 2 # we index 1 spot over (forward 2 back one) 
        self.prevImage()
        
        
        
    def quitEvent(self):
        #Exits gui and automatically saves progress
        self.save()
        self.parent.exitGui()
        
    def sliderBeacon(self): # This function sends a signal to the main loop... 
        self.beacon = True       #that informs it to change the nav slider values.
        
    def configNavSliders(self):  
        trainingIndex = self.i # we will make them the same...
        imgHeight = self.parent.imageDisplay.imageHeight # this will allow us to retrieve the corresponding image size metadata...
        imgWidth = self.parent.imageDisplay.imageWidth # that is necessary in augmenting the sizes of the nav sliders.
        self.parent.lButtons.yNavSlider.configure(to_ = imgHeight)# augment nav sliders
        self.parent.lButtons.xNavSlider.configure(to_ = imgWidth)# !TODO: fix to be more accurate
        
    

In [9]:
inputDir = os.path.relpath('.\pics\\')  # Find input folder
def main():   
    imgList = scrapeDir(inputDir)
    #imgList = "S:\\OSSP-master\\Training"
    imgType = 'pan' #!TODO: fix this to be variable - argparser
    tdsFilename = ".\training_datasets\\icebridge_v5_training_data.h5"
     
    #s = data.availableImages[self.imageIndex]
    #for i in imgList:
    
    rasterList = []
    i = 0 # indexing var for creating list of unique raster data for each image
    while i < len(imgList):
        src_ds2 = gdal.Open(imgList[i], gdal.GA_ReadOnly)
        rasterList.append(src_ds2)
        i += 1
    #print(src_ds2.ReadAsArray())
    
    #print(rasterList)
    
    #Create root
    root = tk.Tk()
    #root.geometry('800x800')
    #root.attributes('-fullscreen', True) #!TODO VVV activate_autorun = autorun_flag insert into TrainingWindow args
      
    #Initialize Gui
    window = TrainingWindow(root, imgList, tdsFilename, imgType, rasterList)
    #print(window.data.getSizeLists())
    #print(src_ds2.RasterXSize)
    #print(window.data.availableImages)
    window.pack(fill='both', expand=True)
    
    root.mainloop()

In [ ]:
main()